In [1]:
pip install praatio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from praatio import textgrid
from pydub import AudioSegment
import os
import random
import shutil
import json
import librosa

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Step 1: text train split based on the original set provided

In [82]:

def test_train_split(source_audio,dest_audio_train,dest_audio_test,source_transcript,dest_transcript_train,dest_transcript_test,num):
 
  src_dir = source_transcript
  #dst_dir = dest_transcript
  arr_dir = [dest_audio_train,dest_audio_test,dest_transcript_train,dest_transcript_test]
  #clearing the test folder for both transcript and audio
  for dest_dir in arr_dir:
    for filename in os.listdir(dest_dir):
        f = os.path.join(dest_dir, filename)
          # checking if it is a file
        if os.path.isfile(f):
            os.remove(f)

  # Get a list of all the file names in the source directory
  files = os.listdir(src_dir)
  print(files)
  # Shuffle the list of file names
  random.shuffle(files)

  # Split the list into two equal parts (pass test file num as num)

  split_index = len(files) // num
  #print(str(len(files)) + "len(files)")
  files_a = files[:split_index]
  files_b = files[split_index:]

  # Copy the files in the first list to the destination directory
  for file_name in files_b:
      src_path = os.path.join(src_dir, file_name)
      dst_path = os.path.join(dest_transcript_train, file_name)
      src_path_audio = os.path.join(source_audio, file_name.replace(".TextGrid",".wav"))
      dst_path_audio = os.path.join(dest_audio_train, file_name.replace(".TextGrid",".wav"))
      #print("dst_path" + dst_path)
      #print("dst_path_audio" + dst_path_audio)
      shutil.copy(src_path, dst_path)
      shutil.copy(src_path_audio, dst_path_audio)


  # Copy the files in the second list to the test directory
  for file_name in files_a:
      src_path = os.path.join(src_dir, file_name)
      dst_path = os.path.join(dest_transcript_test, file_name)
      src_path_audio = os.path.join(source_audio, file_name.replace(".TextGrid",".wav"))
      dst_path_audio = os.path.join(dest_audio_test, file_name.replace(".TextGrid",".wav"))
      #print("dst_path" + dst_path)
      #print("dst_path_audio" + dst_path_audio)
      shutil.copy(src_path, dst_path)
      shutil.copy(src_path_audio, dst_path_audio)



In [83]:
 ### invoke test train split

 #num indicates basically what fraction to go as the test set. I am choosing 1/4 th hence passing 4 as num
num = 4
src_transcript_dir = '/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/transcripts/original'
dst_transcript_train_dir = '/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/transcripts/train'
dst_transcript_test_dir = '/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/transcripts/test'

##splitting audio for train and test
src_audio_dir = '/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/audio/original'
dst_audio_train_dir = '/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/audio/train'
dst_audio_test_dir = '/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/audio/test'

test_train_split(src_audio_dir,dst_audio_train_dir,dst_audio_test_dir,src_transcript_dir,dst_transcript_train_dir,dst_transcript_test_dir,num)

['app_3562_5123_phnd_fin.TextGrid', 'app_1977_0041_phnd_cc-moe.TextGrid', 'app_0030_0001_phnd_cc-res.TextGrid', 'app_3410_4819_phnd_fin.TextGrid', 'app_0404_0004_phnd_cc-hol.TextGrid', 'app_3464_4928_phnd_fin.TextGrid', 'app_0493_0001_phnd_cc-res.TextGrid', 'app_3660_5319_phnd_neg.TextGrid', 'app_0322_0001_phnd_cc-hot.TextGrid', 'app_1854_0032_phnd_cc-moe.TextGrid', 'app_3392_4784_phnd_fin.TextGrid', 'app_0297_0001_phnd_cc-hot.TextGrid', 'app_1622_0018_phnd_cc-hdb.TextGrid', 'app_3728_5455_phnd_fin.TextGrid', 'app_3401_4801_phnd_fin.TextGrid', 'app_1953_0014_phnd_cc-moe.TextGrid', 'app_0268_3536_phnd_cc-res.TextGrid', 'app_3631_5262_phnd_fin.TextGrid', 'app_3518_5035_phnd_fin.TextGrid', 'app_1523_0003_phnd_cc-hdb.TextGrid', 'app_0321_0001_phnd_cc-res.TextGrid', 'app_1370_5740_phnd_cc-msf.TextGrid', 'app_0486_0024_phnd_cc-hot.TextGrid', 'app_2055_7010_phnd_cc-msf.TextGrid', 'app_0267_3534_phnd_cc-res.TextGrid', 'app_0554_0022_phnd_cc-hol.TextGrid', 'app_3091_4181_phnd_fin.TextGrid', 'ap

In [ ]:
##Define the tier name, input/out for audio and textgrid files

tier_name = "channel1_S1" # Replace this with the actual name of the tier you want to work with

## Input and target for Textgrid and targer .csv files

manifest_path  = "/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/"
input_textgrid_folder = "/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/transcripts"
target_script_dir = '/transcripts_processed'
 
 ## Input and target for audio and target split audio files 
input_audio_folder = "/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/audio"
target_audio_dir = '/audio_processed'

src_transcript_dir = '/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/transcripts/original'
dst_transcript_train_dir = '/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/transcripts/train'
dst_transcript_test_dir = '/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/transcripts/test'
#test_train_split(src_transcript_dir,dst_transcript_dir,num);

##splitting audio for train and test
src_audio_dir = '/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/audio/original'
dst_audio_train_dir = '/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/audio/train'
dst_audio_test_dir = '/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/audio/test'

In [84]:

def extract_intervals(textgrid_file, tier_name):
 tg = textgrid.openTextgrid(textgrid_file,False)
  #print(tg.tierNames)
  #print(tg.tiers)
  # Get the tier you want to work with
 #tier = tg.get_tier_by_name(tier_name)
 tier = tg.getTier(tier_name)
 print(tier.entries)
  # Extract xmin, xmax, and label (text) from the tier's entries
 intervals = [(entry.start, entry.end, entry.label) for entry in tier.entries]
 return intervals


In [85]:
def split_wav_file(input_file, start_time, end_time, output_file):
# Load the input .wav file
    audio = AudioSegment.from_wav(input_file)
    # Convert start and end time to milliseconds
    start_time_ms = start_time * 1000
    end_time_ms = end_time * 1000
    # Slice the audio
    sliced_audio = audio[start_time_ms:end_time_ms]
    # Export the sliced audio to a new .wav file
    #return sliced_audio
    sliced_audio.export(output_file, format="wav")

In [86]:
# Function to build a manifest
def build_manifest(transcripts_path, manifest_path, wav_path):
    audio_paths=[]; durations=[]; texts=[]
    with open(transcripts_path, 'r') as fin:
        with open(manifest_path, 'a') as fout:
            for line in fin:
                # Lines look like this:
                # <s> transcript </s> (fileID)
                #transcript = line[: line.find('(')-1].lower()
                #print("line after processing step 1" + transcript)
                transcript = line.replace('<s>', '').replace('</s>', '')
              # print("line after processing step 2" + transcript)
                transcript = transcript.strip()
                #print("line after processing step 3" + transcript)


                file_id = line[line.find('(')+1 : -2]  # e.g. "cen4-fash-b"
                #audio_path = os.path.join(
                #    data_dir, wav_path,
                #   file_id[file_id.find('-')+1 : file_id.rfind('-')],
                #   file_id + '.wav')
                #print("line = " + line)
            
                #print("transcript = " + transcript)
              
                duration = librosa.core.get_duration(filename=wav_path)

                #audio_paths.append(audio_path)
                audio_paths.append(wav_path)
                durations.append(duration)
                texts.append(transcript)
                # Write the metadata to the manifest
                metadata = {
                    "audio_filepath": wav_path,
                    "duration": duration,
                    "text": transcript
                }
                json.dump(metadata, fout)
                fout.write('\n') 
    return audio_paths, durations, texts

In [87]:
def cleanupfolders():
    #%cd /content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject
    textgrid_folder_train = "/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/transcripts_processed/train"
    audio_folder_train = "/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/audio_processed/train"
    textgrid_folder_test = "/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/transcripts_processed/test"
    audio_folder_test = "/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/audio_processed/test"
    adhoc_folder = "/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/transcripts_processed"
    adhoc_folder2 = "/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/audio_processed"
    folders = [adhoc_folder,adhoc_folder2,textgrid_folder_train,audio_folder_train,textgrid_folder_test,audio_folder_test]
    for folder in folders:
      print(folder)
      for filename in os.listdir(folder):
        i = 0
        #print(filename)
        f = os.path.join(folder, filename)
        # checking if it is a file
        if os.path.isfile(f):
            #print(f)
            #tg = textgrid.openTextgrid(f,False)
            os.remove(f)

    for filename in os.listdir(textgrid_folder_train):
        i = 0
        f = os.path.join(textgrid_folder_train, filename)
        # checking if it is a file
        if os.path.isfile(f):
            #print(f)
            #tg = textgrid.openTextgrid(f,False)
            #print(filename)
            #if  '.txt' in filename:
            os.remove(f)


In [ ]:
##claen up existing train and test processed folders if you are going to pre-process the data
cleanupfolders()

/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/transcripts_processed
/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/audio_processed
/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/transcripts_processed/train
/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/audio_processed/train
/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/transcripts_processed/test
/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/audio_processed/test


In [88]:

def dataprocessing(input_textgrid_folder,type):
 
  manifest_file = ""
  if type == "train":
    manifest_file = "manifest_train.json"
  else:
     manifest_file = "manifest_test.json"
  manifest_file = manifest_path + manifest_file

  if os.path.isfile(manifest_file):
    #if  '.txt' in filename:
    os.remove(manifest_file)
    print("file removed")
 # m_file  = open(manifest_file,"w")
  if type=="train":
      arr_dir = [dst_transcript_train_dir,dst_audio_train_dir]
  else:
      arr_dir = [dst_transcript_test_dir,dst_audio_test_dir]

  for filename in os.listdir(input_textgrid_folder):
      i = 0
      f = os.path.join(input_textgrid_folder, filename)
      # checking if it is a file
      if os.path.isfile(f):
          #print(f)
          tg = textgrid.openTextgrid(f,False)
          #audiofile = os.path.join(input_audio_folder, f.replace(".TextGrid",".wav"))
          audiofile = f.replace(".TextGrid",".wav")
          audiofile = audiofile.replace("/transcripts","/audio")
          #print("audio file = " + audiofile)
          #print("transcript file = " + f)
          #audiofile = f.replace(subst,"-"+str(i)+'.txt')
          #print(tg.tierNames)
          #print(tg.tiers)
            # Get the tier you want to work with
          #tier = tg.get_tier_by_name(tier_name)
          tier = tg.getTier(tier_name)
          #print(tier.entries)
            # Extract xmin, xmax, and label (text) from the tier's entries
          intervals = [(entry.start, entry.end, entry.label) for entry in tier.entries]
          #intervals = extract_intervals(textgrid_file, tier_name)
          i = 0
          for xmin, xmax, text in intervals:
            #print(f"Start: {xmin}, End: {xmax}, Text: {text}")
            #f = f+ str(i)
            i = i+1
            ln =len(f)-9
            #print('subst' + str(ln))
            subst = f[ln:ln+9]

            audioln =len(audiofile)-7
            #print('subst' + str(ln))
            audiosubst = audiofile[audioln:audioln+3]
            #print('audiosubst : ' + audiosubst)

            fname = f.replace(subst,"-"+str(i)+'.txt')
            audioName = audiofile.replace(audiosubst,audiosubst+"-"+str(i))
            #fname = fname.replace(subst,'_'+str(i)+ '.txt')
            #print('audioName' + audioName )
            #/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/transcripts/original/app_0030_0001_phnd_cc-res.TextGrid
            #target_script_dir = '/transcripts_processed'
 
 ## Input and target for audio and target split audio files 
#input_audio_folder = "/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/audio"
#target_audio_dir = '/audio_processed'
            fname = fname.replace("/transcripts/original/",target_script_dir+"/"+type+"/")
            audioName = audioName.replace("/audio/original/",target_audio_dir+"/"+type+"/")
            #print('fname' + fname )
            #print('audioName' + audioName )
            if  (text != "" and  text != "<S>"):

              file  = open(fname,"w")
              #with open(fname,"w") as file:
              #print('fname=' + fname)
              file.write(text)
              file.close()
              split_wav_file(audiofile, xmin, xmax, audioName)
              #if os.path.isfile(f):
               # print(f)
            #tg = textgrid.openTextgrid(f,False)
            #print(filename)
               
              #m_string = str(i)+","+fname+","+audioName+"\n"
              #m_file.write(m_string)
              build_manifest(fname,manifest_file,audioName)


In [89]:
def count_files(path):
    count = 0
    for root, dirs, files in os.walk(path):
        count += len(files)
    return count


In [90]:
def getFileStatistics():
  %cd  "/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject"
  folders = []
  for root, dirs, files in os.walk("Data"):
          for dir in dirs:
              folder_path = os.path.join(root, dir)
              if ".ipynb_checkpoints" not in folder_path:
                  count = count_files(folder_path)
                  print("folder_path = " + folder_path + " has  " + str(count) + " files ")
              #folders.append(folder_path)
    



In [91]:
# get file statistics before file processing

getFileStatistics()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-91-93626bd73479>", line 3, in <module>
    getFileStatistics()
  File "<ipython-input-90-65c4fd416a31>", line 2, in getFileStatistics
    get_ipython().run_line_magic('cd', ' "/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject"')
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 2314, in run_line_magic
    result = fn(*args, **kwargs)
  File "<decorator-gen-85>", line 2, in cd
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/magics/osm.py", line 334, in cd
    oldcwd = os.getcwd()
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, anoth

OSError: ignored

In [ ]:
## split each file under test/trainbased on the duration of the conversation and prepare a manifest
dataprocessing(src_transcript_dir,"test")


file removed


<ipython-input-46-aedf47931268>:26: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.core.get_duration(filename=wav_path)


In [ ]:
dataprocessing(src_transcript_dir,"train")

file removed


<ipython-input-46-aedf47931268>:26: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.core.get_duration(filename=wav_path)


In [11]:
folder_path = "/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data"
output_path = "/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data"
!zip -r "/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data.zip" "/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data"

Streaming output truncated to the last 5000 lines.
  adding: content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/transcripts_processed/test/app_0297_0001_phnd_cc-hot-110.txt (deflated 34%)
  adding: content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/transcripts_processed/test/app_0297_0001_phnd_cc-hot-111.txt (deflated 43%)
  adding: content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/transcripts_processed/test/app_0297_0001_phnd_cc-hot-112.txt (stored 0%)
  adding: content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/transcripts_processed/test/app_0297_0001_phnd_cc-hot-113.txt (stored 0%)
  adding: content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/transcripts_processed/test/app_0297_0001_phnd_cc-hot-114.txt (deflated 35%)
  adding: content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/transcripts_processed/test/app_0297_0001_phnd_cc-hot-115.txt (stored 0%)
  adding: content/drive/MyDrive/Colab No

In [9]:
import zipfile
import os

#def zip_folder(folder_path, output_path):
"""
Compresses the folder at folder_path and saves the resulting ZIP file at output_path.
"""
folder_path = "/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/transcripts_processed/train"
output_path = "/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data"
with zipfile.ZipFile(folder_path, 'w', zipfile.ZIP_DEFLATED) as zipObj:
      
      # iterate over all the files and folders in the folder
      for foldername, subfolders, filenames in os.walk(folder_path):
          
          # add the current folder to the zip file
          zipObj.write(foldername)
          
          # add all the files in the current folder to the zip file
          for filename in filenames:
              file_path = os.path.join(foldername, filename)
              zipObj.write(file_path)

# Example usage
#zip_folder("/path/to/folder", "/path/to/output.zip")

IsADirectoryError: ignored

BeLOW are one time scripts used to copy the files over or to dele

In [ ]:
import shutil
import os

# Path of the source directory
src_dir = "/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/audio"
# Path of the destination directory
dst_dir = "/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/audio/train"

# Get the list of all files in the source directory
files = os.listdir(src_dir)

# Iterate over all the files and copy them to the destination directory
for file_name in files:
    
      # Construct the full file path
      src_file = os.path.join(src_dir, file_name)
      dst_file = os.path.join(dst_dir, file_name)

      # Copy the file
      #print(src_file)
      #print(dst_file)
      if os.path.isfile(src_file):
        print("if there is a need to copy enable the below line")
        ##shutil.copy(src_file, dst_file)
      #os.remove(src_file)

In [ ]:
##Renane old files

old_audio = "/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/audio"
old_speech = "/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/transcripts"
counter = 1
for filename in os.listdir(old_audio):
        filenamenew = filename.replace(".wav.wav",".wav")
        os.rename(os.path.join(old_audio, filename), os.path.join(old_audio, filenamenew))
        #os.rename(filename, filenamenew)
        #counter += 1

for filename in os.listdir(old_speech):
        filenamenew = filename.replace(".wav.TextGrid",".TextGrid")
        os.rename(os.path.join(old_speech, filename), os.path.join(old_speech, filenamenew))
        #os.rename(filename, filenamenew)
        #counter += 1
      

In [ ]:
import zipfile
 # Replace this with the path to your folder
#file_name = "training_pack1.zip"        # Replace this with the name of your file

# Create the file path by joining the folder path and file name



zip_file_transcript = "/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/transcripts.zip"
zip_file_transcript_folder = "/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/transcripts"
zip_folder = "/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data"
sample = "/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/sample.zip"

#zip_path = os.path.join(zip_folder, file_name)




with zipfile.ZipFile(zip_file_transcript, "r") as zip_ref:
    zip_ref.extractall(zip_file_transcript_folder)

    #file_list = zip_ref.namelist()
    #print(file_list)

